# Decorators
It takes a function as an argumnt and modifies the beaviour of the input function

In [91]:
# Normal function in python

def add_one(n):
    return n+1

print(add_one(3))

4


### First-Class Objects
In Python, functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object (string, int, float, list, and so on). Consider the following three functions:


In [92]:
def say_hello(name):
    return f"Hello {name}"

def greet_bob(greater_fun):
    return greater_fun("Bob")

greet_bob(say_hello)

'Hello Bob'

### Inner Functions
It’s possible to define functions inside other functions. Such functions are called inner functions. Here’s an example of a function with two inner functions:

In [93]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


In [94]:
# Functions defined inside the parent function are defined in local scope ,
# therefore they can't be accessed outside of the class , and are called only when parent function is called
second_child()

NameError: name 'second_child' is not defined

### Simple Decorators

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happning before function call")
        func()
        print("Something is happning after function call")
    return wrapper

def say_hi():
    print("Hi")

say_hi1 = my_decorator(say_hi)   # Applying decorator on 'say_hi()' function

say_hi1()

Something is happning before function call
Hi
Something is happning after function call


In [ ]:
say_hi1

# Decorators wrap a function, modifying its behavior.

<function __main__.my_decorator.<locals>.wrapper()>

In [ ]:
# In this example decorated code runs only during the day
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7<= datetime.now().hour <22:
            func()
        else:
            pass
    return wrapper


def say_hi():
    print("say_hi")

say_hi1 = not_during_the_night(say_hi)

say_hi1()

say_hi


### Syntactic Sugar!

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator  # Applying the decorator on a function (better way)
def say_hi():
    print("hiiii")

say_hi()

Something is happening before the function is called.
hiiii
Something is happening after the function is called.


### Decorating Functions With Arguments

In [ ]:
from Decorators.decorators import do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

greet("world")

TypeError: do_twice.<locals>.wrapper_do_twice() takes 0 positional arguments but 1 was given

The problem is that the inner function wrapper_do_twice() does not take any arguments, but name="World" was passed to it. You could fix this by letting wrapper_do_twice() accept one argument, but then it would not work for the say_whee() function you created earlier.

The solution is to use *args and **kwargs in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. Rewrite decorators.py as follows:



```python
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice
```


In [ ]:
from Decorators.decorators_new import do_thrice

@do_thrice
def greet(name):
    print(f"Hello {name}")

greet("world")

Hello world
Hello world
Hello world


In [ ]:
@do_thrice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

hi_rohit = return_greeting("rohit")
print(hi_rohit) # Here decorater returned nothing


Creating greeting
Creating greeting
Creating greeting
Hi rohit


### Function identity

In [ ]:
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [ ]:
print.__name__

'print'

In [ ]:
help(print)

Help on built-in function print in module builtins:

print(*args, sep=' ', end='\n', file=None, flush=False)
    Prints the values to a stream, or to sys.stdout by default.
    
    sep
      string inserted between values, default a space.
    end
      string appended after the last value, default a newline.
    file
      a file-like object (stream); defaults to the current sys.stdout.
    flush
      whether to forcibly flush the stream.



In [ ]:
# The introspection works for functions you define yourself as well:
print(add_one)
print(add_one.__name__)
print(help(add_one))


<function add_one at 0x7f8af4006700>
add_one
Help on function add_one in module __main__:

add_one(n)

None


In [ ]:
# Similarly is should work after applying decorator
print(greet)
print(greet.__name__)  # here function has lost its identity
print(help(greet))  # here function has lost its identity

<function do_thrice.<locals>.wrapper_do_thrice at 0x7f8af415f740>
wrapper_do_thrice
Help on function wrapper_do_thrice in module Decorators.decorators_new:

wrapper_do_thrice(*args, **kwargs)

None


To retaion the identity of original function even after applying decorator, we should add the\
the following in the decorator function

```python
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice
```

In [ ]:
# Similarly is should work after applying decorator

from Decorators.decorators_new import do_thrice2

@do_thrice2
def new_greet(name):
    print(f"hello {name}")


<function __main__.new_greet(name)>

In [ ]:
print(new_greet)  # here identity not lost
print(new_greet.__name__)
print(help(new_greet))

<function new_greet at 0x7f8af3074e00>
new_greet
Help on function new_greet in module __main__:

new_greet(name)

None


### Real World Examples
This formula is a good boilerplate template for building more complex decorators

In [ ]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator


In [97]:
# Creating timer decorator

import functools
import time

def timer(func):
    """Print the runtime of the decortated function"""
    @functools.wraps(func)
    def wrapper_timer(*args,**kwargs):
        start_time = time.perf_counter() #1
        value = func(*args,**kwargs)
        end_time = time.perf_counter()  #1
        run_time = end_time-start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer




In [ ]:
@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])
    # return "finished "


print(waste_some_time(1))

Finished 'waste_some_time' in 0.0013 secs
None


In [ ]:
print(waste_some_time(1000))

Finished 'waste_some_time' in 0.7958 secs
None


### Debugging Code using decorator

In [96]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrpapper_debug(*args,**kwargs):
        args_repr = [repr(a) for a in args]                         #1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]     #2
        signature = ", ".join(args_repr+kwargs_repr)                #3
        print(f"calling {func.__name__}({signature})")
        value = func(*args,**kwargs)
        print(f"{func.__name__!r} retuened {value!r}")              #4
        return value
    return wrpapper_debug


# 1.Create a list of the positional arguments. Use repr() to get a nice string representing each argument.

# 2.Create a list of the keyword arguments. The f-string formats each argument as key=value where 
# the !r specifier means that repr() is used to represent the value.

# 3.The lists of positional and keyword arguments is joined together to one signature string with each argument separated by a comma.

# 4.The return value is printed after the function is executed.


In [ ]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"


In [ ]:
make_greeting("amit")

calling make_greeting('amit')
'make_greeting' retuened 'Howdy amit!'


'Howdy amit!'

In [ ]:
make_greeting("Rohit",23)

calling make_greeting('Rohit', 23)
'make_greeting' retuened 'Whoa Rohit! 23 already, you are growing up!'


'Whoa Rohit! 23 already, you are growing up!'

In [ ]:
import math


# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

approximate_e(2)

calling factorial(0)
calling factorial(0)
calling factorial(0)
'factorial' retuened 1
'factorial' retuened 1
'factorial' retuened 1
calling factorial(1)
calling factorial(1)
calling factorial(1)
'factorial' retuened 1
'factorial' retuened 1
'factorial' retuened 1


2.0

### Slowing Down Code


In [ ]:
import functools
import time

def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args,**kwargs):
        time.sleep(1)  # sleep for 1 sec
        return func(*args,**kwargs)
    return wrapper_slow_down

@slow_down
def count_down(from_number):
    if from_number < 1 :
        print("Lift off!")
    else:
        print(from_number)
        count_down(from_number-1)
    

count_down(5)

5
4
3
2
1
Lift off!


### Registering plugins
This can be used, for instance, to create a light-weight plug-in architecture:

In [ ]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_aweseome(name):
    return f"{name}, you are awesome!"


def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)


PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_aweseome': <function __main__.be_aweseome(name)>}

In [ ]:
randomly_greet("amit")

Using 'be_aweseome'


'amit, you are awesome!'

In [ ]:
# Global variables can also be accessed by calling globals()
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import random\nPLUGINS = dict()\n\ndef register(func):\n    """Register a function as a plug-in"""\n    PLUGINS[func.__name__] = func',
  'import random\nPLUGINS = dict()\n\ndef register(func):\n    """Register a function as a plug-in"""\n    PLUGINS[func.__name__] = func\n    return func\n\n@register\ndef say_hello(name):\n    return f"Hello {name}"\n\ndef be_aweseome(name):\n    return f"{name}, you are awesome!"\n\n\ndef randomly_greet(name):\n    greeter, greeter_func = random.choice(list(PLUGINS.items()))\n    print(f"Using {greeter!r}")\n    return greeter_func(name)',
  'randomly_greet("amit")',
  'randomly_greet("amit")',
  'randomly_greet("amit")',
  'randomly_greet("amit")',
  'randomly_greet("amit")'

### Fancy Decorators
So far we saw, simple decorators, there are more ways to create decorators

- Decorators on classes
- Several decorators on one function
- Decorators with arguments
- Decorators that can optionally take arguments
- Stateful decorators
- Classes as decorators


### Decorating classes
We can decorate the methods of a class

 Some commonly used decorators that are even built-ins in Python are @classmethod, @staticmethod, and @property. The @classmethod and @staticmethod decorators are used to define methods inside a class namespace that are not connected to a particular instance of that class. The @property decorator is used to customize getters and setters for class attributes. Expand the box below for an example using these decorators.

#### Example of some build-in decorators

In [ ]:
# The following definition of a Circle class uses the @classmethod, @staticmethod, and @property decorators:

class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535


In above class:

- .cylinder_volume() is a regular method.
- .radius is a mutable property: it can be set to a different value. However, by defining a setter method, we can do some error testing to make sure it’s not set to a nonsensical negative number. Properties are accessed as attributes without parentheses.
- .area is an immutable property: properties without .setter() methods can’t be changed. Even though it is defined as a method, it can be retrieved as an attribute without parentheses.
- .unit_circle() is a class method. It’s not bound to one particular instance of Circle. Class methods are often used as factory methods that can create specific instances of the class.
- .pi() is a static method. It’s not really dependent on the Circle class, except that it is part of its namespace. Static methods can be called on either an instance or the class.

The Circle class can for example be used as follows:

In [ ]:
c = Circle(5)
c.radius

5

In [ ]:
c.area # it is a property not a method, if it was a method then we had to use 'c.area()'

78.5398163375

In [ ]:
print(type(c.area)) # 
print(type(c.cylinder_volume)) # 

<class 'float'>
<class 'method'>


In [ ]:
c.cylinder_volume(height=8)

628.3185307

In [ ]:
c.radius = 5
c.radius = -1  # raise rerror on setting negative values

ValueError: Radius must be positive

In [ ]:
c.pi()

3.1415926535

In [ ]:
c2 = Circle.unit_circle()
c2.radius

1

In [ ]:
Circle.pi()

3.1415926535

Let’s define a class where we decorate some of its methods using the @debug and @timer decorators from earlier:

In [98]:
class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])


In [99]:
tw = TimeWaster(100)

calling __init__(<__main__.TimeWaster object at 0x7f0d2e0e85d0>, 100)
'__init__' retuened None


In [100]:
tw.waste_time(999)

Finished 'waste_time' in 0.0184 secs


In [103]:
# The other way to use decorators on classes is to decorate the whole class.
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str

#It could have done the decoration by writing PlayingCard = dataclass(PlayingCard).

<class '__main__.PlayingCard'>


In [104]:
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

Decorating a class does not decorate its methods. Recall that `@timer` is just shorthand for `TimeWaster = timer(TimeWaster).`

Here, `@timer` only measures the time it takes to instantiate the class:

In [105]:
tw = TimeWaster(1000)

Finished 'TimeWaster' in 0.0000 secs


In [107]:
tw.waste_time(999)

### Nesting decorator

In [116]:
from Decorators.decorators_new import do_thrice
@debug
@do_thrice
@timer
def greet(name):
    print(f"Hello {name}")


greet("Rohan")

calling wrapper_do_thrice('Rohan')
Hello Rohan
Finished 'greet' in 0.0000 secs
Hello Rohan
Finished 'greet' in 0.0000 secs
Hello Rohan
Finished 'greet' in 0.0000 secs
'wrapper_do_thrice' retuened None


### Decorators With Arguments

In [122]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args,**kwargs):
            for _ in range(num_times):
                value = func(*args,**kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat



@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")


greet("there")

Hello there
Hello there
Hello there
Hello there
